<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [363]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception
from keras.preprocessing.image import ImageDataGenerator


In [364]:
#git clone https://github.com/mohanrajmit/Transfer-Learning.git

In [365]:
ls

model/     test_dataset/  train_Healthy/           wewe/
README.md  train/         training_dataset/
test/      train_Defect/  Transfer_Learning.ipynb


In [366]:
import os 
a = os.getcwd()
print(a)

/home/gaurav/data/inception/Transfer-Learning


In [367]:
'''
path_Defect_input  ="/home/gaurav/data/inception/Transfer-Learning/train_Defect"
path_Defect_output ="/home/gaurav/data/inception/Transfer-Learning/training_dataset/Defect/"
path_healthy_input ="/home/gaurav/data/inception/Transfer-Learning/train_Healthy"
path_healthy_output = "/home/gaurav/data/inception/Transfer-Learning/training_dataset/Healthy/"


train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip= True)
i = 1
for batch1 in train_datagen.flow_from_directory(path_healthy_input, batch_size=10,save_to_dir=path_healthy_output,save_prefix='parts', save_format='jpg'):
    i += 1
    if i > 10:
      break  # otherwise the generator would loop indefinitely.
    
j = 1 
for batchk in train_datagen.flow_from_directory(path_Defect_input, batch_size=10,save_to_dir=path_Defect_output,save_prefix='parts', save_format='jpg'):
    j += 1
    if j > 10:
      break  # otherwise the generator would loop indefinitely.
'''

'\npath_Defect_input  ="/home/gaurav/data/inception/Transfer-Learning/train_Defect"\npath_Defect_output ="/home/gaurav/data/inception/Transfer-Learning/training_dataset/Defect/"\npath_healthy_input ="/home/gaurav/data/inception/Transfer-Learning/train_Healthy"\npath_healthy_output = "/home/gaurav/data/inception/Transfer-Learning/training_dataset/Healthy/"\n\n\ntrain_datagen = ImageDataGenerator(\n        rescale=1./255,\n        rotation_range=20,\n        shear_range=0.2,\n        horizontal_flip=True,\n        vertical_flip= True)\ni = 1\nfor batch1 in train_datagen.flow_from_directory(path_healthy_input, batch_size=10,save_to_dir=path_healthy_output,save_prefix=\'parts\', save_format=\'jpg\'):\n    i += 1\n    if i > 10:\n      break  # otherwise the generator would loop indefinitely.\n    \nj = 1 \nfor batchk in train_datagen.flow_from_directory(path_Defect_input, batch_size=10,save_to_dir=path_Defect_output,save_prefix=\'parts\', save_format=\'jpg\'):\n    j += 1\n    if j > 10:\n

In [368]:
x_train = []
# Load the training data set by looping over every image file
for image_file in Path("/home/gaurav/data/inception/Transfer-Learning/training_dataset").glob("**/*.jpg"):
    #print(image_file)
    
    
    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)


print(len(x_train))

215


In [369]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

In [370]:
# Add a label for this image. If it was a Defect, label it 0. If it was a Healthy, label it 1.
y_train = []
path = "/home/gaurav/data/inception/Transfer-Learning/training_dataset/"
classes = ["Defect","Healthy"]
for i in classes:
    if i == "Defect":
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_train.append(0)
    else:
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_train.append(1)
print(len(y_train))

215


In [371]:
# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))
x = feature_extractor
#print(x)


In [372]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "/home/gaurav/data/inception/Transfer-Learning/model/x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "/home/gaurav/data/inception/Transfer-Learning/model/y_train.dat")


['/home/gaurav/data/inception/Transfer-Learning/model/y_train.dat']

In [373]:
# Load data set of extracted features
x_train = joblib.load("/home/gaurav/data/inception/Transfer-Learning/model/x_train.dat")
y_train = joblib.load("/home/gaurav/data/inception/Transfer-Learning/model/y_train.dat")


In [374]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [375]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [376]:
print(len(x_train))
print(len(y_train))

215
215


In [383]:
# Train the model
model.fit(
    x_train,
    y_train,
    validation_split=0.05,
    epochs=10,
    shuffle=True,
    verbose=2
)

# Save the trained model to a file so we can use it to make predictions later
model.save("/home/gaurav/data/inception/Transfer-Learning/model/Defect_healthy_classifier_model.h5")

Train on 204 samples, validate on 11 samples
Epoch 1/15
 - 7s - loss: 0.3298 - acc: 0.8627 - val_loss: 0.3198 - val_acc: 0.8182
Epoch 2/15
 - 0s - loss: 0.3384 - acc: 0.8529 - val_loss: 0.3874 - val_acc: 0.8182
Epoch 3/15
 - 0s - loss: 0.2668 - acc: 0.8922 - val_loss: 0.4955 - val_acc: 0.7273
Epoch 4/15
 - 0s - loss: 0.3083 - acc: 0.8480 - val_loss: 0.4503 - val_acc: 0.7273
Epoch 5/15
 - 0s - loss: 0.2594 - acc: 0.8529 - val_loss: 0.6690 - val_acc: 0.5455
Epoch 6/15
 - 0s - loss: 0.2497 - acc: 0.8971 - val_loss: 0.4925 - val_acc: 0.7273
Epoch 7/15
 - 0s - loss: 0.1977 - acc: 0.9118 - val_loss: 0.4600 - val_acc: 0.7273
Epoch 8/15
 - 0s - loss: 0.1801 - acc: 0.9314 - val_loss: 0.6495 - val_acc: 0.7273
Epoch 9/15
 - 0s - loss: 0.1400 - acc: 0.9657 - val_loss: 0.4593 - val_acc: 0.7273
Epoch 10/15
 - 0s - loss: 0.1729 - acc: 0.9314 - val_loss: 0.5608 - val_acc: 0.7273
Epoch 11/15
 - 0s - loss: 0.1682 - acc: 0.9412 - val_loss: 0.4306 - val_acc: 0.7273
Epoch 12/15
 - 0s - loss: 0.1288 - acc: 

In [384]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [385]:
#Add a label for this image. If it was a Defect, label it 0. If it was a Healthy, label it 1.
y_test = []
path = "/home/gaurav/data/inception/Transfer-Learning/test_dataset"
classes = ["Defect","Healthy"]
for i in classes:
    if i == "Defect":
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_test.append(0)
    else:
        new_path = os.path.join(path,i)
        y = os.listdir(new_path)
        for i in y:
            y_test.append(1)


In [386]:
# Empty lists to hold the images and labels for each each image
x_test = []

# Load the test data set by looping over every image file
for image_file in Path("/home/gaurav/data/inception/Transfer-Learning/test_dataset").glob("**/*.jpg"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)
    
# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

print(x_test.shape)

(28, 73, 73, 3)


In [387]:
# Load our trained classifier model
model = load_model("/home/gaurav/data/inception/Transfer-Learning/model/Defect_healthy_classifier_model.h5")
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

print(features_x.shape)


(28, 3, 3, 2048)


In [382]:
from sklearn.metrics import accuracy_score


# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)
#print(predictions)

# If the model is more than 50% sure the object is a Defect, call it a Defect.
# Otherwise, call it "Healthy".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions,labels=[0,1]).ravel()
print("True negative",tn)
print("False positive",fp)
print("False negative",fn)
print("True Positive",tp)

score = accuracy_score(y_test, predictions)
print("Accuracy score percent is :",score)




True negative 9
False positive 5
False negative 2
True Positive 12
Accuracy score percent is : 0.75
